In [104]:
import os
import re
import itertools

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import nltk

Outline:
- read in data
- preprocess 
- model
blabla
sources:
https://www.kaggle.com/dskswu/topic-modeling-bert-lda - topic modeling bert+lda

In [2]:
home = 0
if home:
    infiledir = r"C:\Users\Luzia T\UCL\WorkingFromHome\Possible_online_studies\NLP_expressive_writing\analysis\Processed_2"
    rand_dir = r"C:\Users\Luzia T\UCL\WorkingFromHome\Possible_online_studies\NLP_expressive_writing\analysis"
else:
    infiledir = r"P:\EW_analysis\analysis\Processed_2"
    writing_dir = r"P:\EW_analysis\analysis\writing\writing_data"



In [98]:
import os

for dirname, _, filenames in os.walk(writing_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


ew_files = [f for f in filenames if 'T1' in f]
ewre_files = [f for f in filenames if 'T2' in f]
ctr_files = [f for f in filenames if 'T3' in f]


P:\EW_analysis\analysis\writing\writing_data\Writing_T1_Day1.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T1_Day2.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T1_Day3.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T1_Day4.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T2_Day1.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T2_Day2.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T2_Day3.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T2_Day4.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T3_Day1.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T3_Day2.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T3_Day3.csv
P:\EW_analysis\analysis\writing\writing_data\Writing_T3_Day4.csv


In [100]:
def get_diaries(directory,filenames,group_name):
    all_files = []
    for i,name in enumerate(filenames):
        diary_df = pd.read_csv(os.path.join(directory,name),skiprows = [0,2])
        # retain only records of people who finished the survey
        diary_df = diary_df.loc[diary_df.Progress==100,:]
        # what day of writing were they on
        diary_df['day'] = i+1
        diary_df['Group'] = group_name
        writing_col = diary_df.filter(like = 'Please use the box below').columns[0]
        diary_df = diary_df.rename(columns = {writing_col: 'writing'})
        all_files.append(diary_df)
    return all_files

def read_files(infiles):
    all_df = pd.concat([pd.DataFrame(infiles[0]), pd.DataFrame(infiles[1]),pd.DataFrame(infiles[2]),pd.DataFrame(infiles[3])])
    return all_df

In [101]:

all_ew = get_diaries(writing_dir, ew_files,'EW')
ew_df = read_files(all_ew)
all_ewre = get_diaries(writing_dir, ewre_files,'EWRE')
ewre_df = read_files(all_ewre)
all_ctr = get_diaries(writing_dir, ctr_files,'CTR')
ctr_df = read_files(all_ctr)
writing_df = pd.concat([ew_df,ewre_df,ctr_df])

# save if needed
save = 0
if save:
    writing_df.to_csv(os.path.join(writing_dir, 'writing_df.csv'))

Let's read in the file holding the complete data set and the data from each day of writing.

In [5]:
all_dat_df = pd.read_csv(os.path.join(infiledir, 'all_dat_df.csv'))

Utils

In [113]:

from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.tokenize import word_tokenize
import pkg_resources
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from symspellpy import SymSpell, Verbosity


sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# normalization
def func_norm(s):
    """
    Some basic normalisation.
    Params: input string
    Out: normalised string.
    
    """
    s = s.lower() # lower case
    # letter repetition (>2)
    s  = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # non word repetition
    s = s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # noise text
    s = re.sub(r' ing ', ' ', s)
    # phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)

    return s.strip()

def func_punc(w_list):
    """
    w_list: word list to be processed
    returns w_list with punctuation and number filter out
    """
    return [word for word in w_list if word.isalpha()]

def func_stopf(w_list):
    """
    filter stop words
    in:  word list
    out: word list w/o stop words
    """
    stop_words = set(stopwords.words('english'))
    w_list  = [f for f in w_list if f not in stop_words]

# stemming
pstem = PorterStemmer()

def func_stem(w_list):
    """
    stem word list
    in: word list for stemming
    out: stemmed word list 
    """
    sw_list = [pstem.stem(w) for w in w_list]
    return sw_list

# selecting nouns
def func_noun(w_list):
    """
    w_list: word list to be processed
    return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']

# spell checker/typo correction
def func_spell(w_list):
    """
    w_list: word list to be processed
    return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            pass
    return w_list_fixed

def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw texts
    param rw: text to be procssed
    return: sentence level pre-processed text
    """
    s = func_norm(rw)

    return s



def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = func_punc(w_list)
    w_list = func_noun(w_list)
    w_list = func_spell(w_list)
    w_list = func_stem(w_list)
    w_list = func_stopf(w_list)

    return w_list

In [ ]:
def preprocess(docs):
    """
    Preprocess the data
    """

    print('Preprocessing raw texts ...')
    n_docs = len(docs)
    sentences = []  # sentence level preprocessed
    token_lists = []  # word level preprocessed
    idx_in = []  # index of sample selected
    samp = np.random.choice(n_docs, samp_size)
    for i, idx in enumerate(samp):
        sentence = preprocess_sent(docs[idx])
        token_list = preprocess_word(sentence)
        if token_list:
            idx_in.append(idx)
            sentences.append(sentence)
            token_lists.append(token_list)
        print('{} %'.format(str(np.round((i + 1) / len(samp) * 100, 2))), end='\r')
    print('Preprocessing raw texts. Done!')
    return sentences, token_lists, idx_in

In [ ]:
sent,tok, idx_in = preprocess(writing_df.writing)

In [ ]:
def main():
    
    
    method = "LDA_BERT"
    samp_size = 51000
    ntopic = 10
    
    #parser = argparse.ArgumentParser(description='contextual_topic_identification tm_test:1.0')

    #parser.add_argument('--fpath', default='/kaggle/working/train.csv')
    #parser.add_argument('--ntopic', default=10,)
    #parser.add_argument('--method', default='TFIDF')
    #parser.add_argument('--samp_size', default=20500)
    
    #args = parser.parse_args()

    data = writing_df.writing #pd.read_csv('/kaggle/working/train.csv')
    #data = data.fillna('')  # only the comments has NaN's
    rws = data.abstract
    sentences, token_lists, idx_in = preprocess(rws, samp_size=samp_size)
    # Define the topic model object
    #tm = Topic_Model(k = 10), method = TFIDF)
    tm = Topic_Model(k = ntopic, method = method)
    # Fit the topic model by chosen method
    tm.fit(sentences, token_lists)
    # Evaluate using metrics
    with open("/kaggle/working/{}.file".format(tm.id), "wb") as f:
        pickle.dump(tm, f, pickle.HIGHEST_PROTOCOL)

    print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
    print('Silhouette Score:', get_silhouette(tm))
    # visualize and save img
    visualize(tm)
    for i in range(tm.k):
        get_wordcloud(tm, token_lists, i)
        